In [1]:
import pandas as pd
import geopandas as gp
import numpy as np
import datetime
import subprocess
import ijson
import json
from subprocess import getoutput
from glob import glob
from dotenv import load_dotenv
load_dotenv()
import os 
from pathlib import Path
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch(['https://3d6a9dd50c7c49c9ab5d23b6891bc03e.us-central1.gcp.cloud.es.io:9243'], 
                http_auth=('elastic', 'WMzYk5RXyzE7MRShwPVwHzPX'), timeout=30)

%load_ext autotime

time: 345 µs (started: 2022-01-03 11:37:49 -05:00)


In [2]:
import sys
def sizeof_fmt(size):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(size) < 1024.0:
            return f"{size:3.1f} {unit}B"
        size /= 1024.0
    return f"{size:.1f} YiB"

print(f'TOTAL SIZE: {sizeof_fmt(sum( (sys.getsizeof(value) for value in locals().values() ))):>10}')
for nam, size in sorted(((nam, sys.getsizeof(value)) for nam, value in locals().items()), key= lambda x: -x[1])[:10]:
    print(f'{nam:>20}: {sizeof_fmt(size):>10}')


TOTAL SIZE:    7.2 KiB
       Elasticsearch:    2.0 KiB
                Path:    896.0 B
                  _i:    552.0 B
                 _i1:    552.0 B
                 _i2:    531.0 B
                 _oh:    232.0 B
                 Out:    232.0 B
           getoutput:    136.0 B
                glob:    136.0 B
         load_dotenv:    136.0 B
time: 1.89 ms (started: 2021-12-20 13:11:27 -05:00)


In [3]:
regridcols = ['cbg', 'lat', 'lon', 'func']
parcelNumIndCols = ['parcelNumAgri', 'parcelNumCommer',  
                    'parcelNumInfra', 'parcelNumNull', 
                    'parcelNumResi']

BLOCKGROUP_FIPS_LENGTH = 12
STATE_FIPS_DICT_52 = { '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', 
                      '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', 
                      '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', 
                      '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', 
                      '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', 
                      '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', 
                      '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', 
                      '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', 
                      '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', 
                      '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', 
                      '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', 
                      '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', 
                      '54': 'WV', '55': 'WI', '56': 'WY', '72': 'PR'}

REGRID_BUCKET_DIR = '/home/nhat/regrid-bucket/'
JQ_JSONS_DIR = '/home/nhat/jq-jsons/'

time: 966 µs (started: 2021-12-20 13:11:30 -05:00)


# Update Regrid data / Regrid verse

https://support.regrid.com/articles/verse
- Parcelverse metadata
- The two critical pieces of information in the verse table are filename_stem and last_refresh. Those two fields tell you the name of the dataset for a county and the last time we did a full data pull from the source, usually the county or a designee.
- **To find the latest updates in verse, sort by 'last_refresh' and use the 'filename_stem' column to identify the file.**

In [4]:
verse = gp.read_file('~/regrid-bucket/verse.csv.zip')
verse.shape # 3232
# verse.isnull().sum() # geometry 3232 ALL EMPTY i.e. useless; anyway, better from source: tiger census 
# verse.dtypes # all object types
# verse['county_state'] = verse['county'] + ',' + verse['state'] 
# len(verse['county_state'].unique()) # 3226
verse = verse.replace('', np.nan)
# verse.isnull().sum()

## CLEANING: Drop empty last_refresh / empty table_name
def set_diff(a, b, col):
    print(f"{set(a[col]) - set(b[col])=}")
    print(f"{set(b[col]) - set(a[col])=}")

time: 767 ms (started: 2021-12-20 13:11:32 -05:00)


In [8]:
# table_name <> last_refresh = 166 ;
# total_objects <> filename_stem <> usps_data_date = 165;
    
# verse_refresh = verse.dropna(subset=['last_refresh'])
# verse_table = verse.dropna(subset=['table_name'])
# set_diff(verse_refresh, verse_table)

# verse_objects = verse.dropna(subset=['total_objects'])
# verse_name = verse.dropna(subset=['filename_stem'])
# verse_usps = verse.dropna(subset=['usps_data_date'])
# len(verse_objects), len(verse_name), len(verse_usps) # (3067, 3067, 3067)
# set_diff(verse_objects, verse_name)
# set_diff(verse_usps, verse_name)
# set_diff(verse_objects, verse_usps)

# set_diff(verse_refresh, verse_objects)
# set_diff(verse_table, verse_name)

# ONLY DIFFERENCE: id = 10190
# set(a['id']) - set(b['id'])=set()

# filename_stem = ok_washington_bartlesville, no last_refresh, has 22507 total objects and not found in our boss-geo-data bucket! CAN IGNORE
# set(b['id']) - set(a['id'])={'10190'} 

time: 621 µs (started: 2021-12-15 05:31:58 +00:00)


In [5]:
verse_refresh = verse.dropna(subset=['last_refresh'])
verse_refresh.shape # (3066, 19)
verse_refresh = verse_refresh[['county', 'state', 'geoid', 'last_refresh', 'filename_stem']].copy()
verse_refresh = verse_refresh.rename(columns={'last_refresh': 'refresh', 'filename_stem': 'name'})
verse_refresh.isnull().sum()

county     0
state      0
geoid      0
refresh    0
name       0
dtype: int64

time: 23.4 ms (started: 2021-12-20 13:11:35 -05:00)


## 3232 rows, with 3226 distinct counties, and 3066 counties with parcel data (non-empty table_name or last_refresh)

In [269]:
# verse[verse.duplicated(['county', 'state'], keep=False)].sort_values(['state','county'])
# verse[verse.duplicated(['geoid'], keep=False)].sort_values(['state','county'])

# len(verse['geoid'].unique()) # 3226
# # 3226 geoids from 3232 rows, only valid repeated geoid = 26163 {mi_wayne_detroit and mi_wayne}
# len(verse) - (verse['filename_stem'] == '').sum() # 3067
# len(verse) - (verse['last_refresh'] == '').sum() # 3066
# len(verse) - (verse['table_name'] == '').sum() # 3066

time: 303 µs (started: 2021-12-14 22:09:00 +00:00)


In [270]:
### Extra: insufficient cleaning
# # extraneous_ids = {'760', '1314', '1322', '1324', '10190', }
# rowIds = ['table_name', 'county', 'state']
# extra = pd.DataFrame(data= {('','DeKalb','IL'),
# ('','Iron','MI'),
# ('','Lapeer','MI'),
# ('','Lenawee','MI'),
# ('','Washington','OK')}, columns=rowIds)

# verse_clean = pd.concat([verse, extra]).drop_duplicates(subset=rowIds, keep=False)
# verse_clean.shape # (3227, 20)
# verse_clean

# (verse_clean['path'] == '').sum() # 0
# (verse_clean['geoid'] == '').sum() # 0
# len(verse_clean['geoid'].unique()) # 3226
# # 3226 geoids from 3227 rows, only repeated geoid = 26163 {mi_wayne_detroit and mi_wayne}

# len(verse_clean) - (verse_clean['filename_stem'] == '').sum() # 3066
# len(verse_clean) - (verse_clean['last_refresh'] == '').sum() # 3066
# len(verse_clean) - (verse_clean['table_name'] == '').sum() # 3066

time: 390 µs (started: 2021-12-14 22:09:03 +00:00)


## Regrid parcels: have 3059 zip files i.e. Regrid has 3059 counties with parcel data

In [6]:
# with open('extras/geoid_c_r.json', 'r') as f:
#     countygeoid = json.load(f)
#     countygeoid = pd.DataFrame({'geoid': countygeoid})
countygeoid = pd.read_json('~/extras/geoid_c_r.json', typ='dictionary')
countygeoid = countygeoid.reset_index()
countygeoid = countygeoid.rename(columns = {'index': 'name', 0: 'geoid'})
countygeoid['geoid'] = countygeoid['geoid'].astype(str).str.zfill(5)
# countygeoid # 3059 rows

time: 19.8 ms (started: 2021-12-20 13:11:39 -05:00)


In [7]:
len(set(verse_refresh['name'])) # 3066
len(set(countygeoid['name'])) # 3059
set_diff(verse_refresh, countygeoid, 'name')
# US territories {'vi_st-john', 'gu_guam', 'vi_st-thomas', 'mp_tinian', 'mp_rota', 'vi_st-croix', 'mp_saipan'}

set(a[col]) - set(b[col])={'vi_st-thomas', 'gu_guam', 'mp_rota', 'mp_saipan', 'mp_tinian', 'vi_st-croix', 'vi_st-john'}
set(b[col]) - set(a[col])={'tx_nacogdoches'}
time: 4.63 ms (started: 2021-12-20 13:11:39 -05:00)


## NOTE: rows with filename_stem = "" (or last_refresh = ""): NEW COUNTY, no data from regird (i.e. no zip files deposited into our boss-geo-data bucket)

In [8]:
ver = verse_refresh.reset_index(drop=True)
us_territories_noncoverage= {'vi_st-john', 'gu_guam', 'vi_st-thomas', 'mp_tinian', 'mp_rota', 'vi_st-croix', 'mp_saipan'}
ver = ver[~ver['name'].isin(us_territories_noncoverage)]
ver['refresh'] = pd.to_datetime(ver['refresh'], format='%Y-%m-%d').dt.date
# NOTE: refresh has datetime.date objects https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.date.html
# ver.dtypes

time: 11 ms (started: 2021-12-20 13:11:58 -05:00)


In [22]:
ver.sort_values('refresh').groupby('refresh').size().tail()


refresh
2021-10-26    139
2021-11-09    157
2021-11-23     54
2021-12-05      1
2021-12-14    218
dtype: int64

time: 5.47 ms (started: 2021-12-21 01:24:05 -05:00)


# NEW UPDATE ON DEC 14

In [25]:
datetime.datetime.fromisoformat('2021-11-23').date()

datetime.date(2021, 11, 23)

time: 2.09 ms (started: 2021-12-21 01:34:24 -05:00)


In [11]:
# datetime.date(2021, 12, 15)
today = datetime.datetime.today().date()

# datetime.date(2021, 11, 28)
latest_refresh_date = ver['refresh'].max()

# FOR TESTING // change this afterward
today = datetime.date(2021, 12, 3)

f"{today=}, {latest_refresh_date=}"


'today=datetime.date(2021, 12, 3), latest_refresh_date=datetime.date(2021, 12, 14)'

time: 2.75 ms (started: 2021-12-20 13:12:37 -05:00)


In [27]:
f"date_{today}"

'date_2021-12-03'

time: 1.71 ms (started: 2021-12-21 01:39:28 -05:00)


# MAIN FUNC

In [29]:
county_dfs = None
if latest_refresh_date >= today:
    # NEED UPDATE
    # find counties that is refreshed today or after
    county_dfs = ver.loc[ver['refresh'] >= today]

state_dfs = [x for _, x in county_dfs.groupby('state')]
# county_dfs

time: 4.52 ms (started: 2021-12-21 01:41:31 -05:00)


In [31]:
county_dfs

,county,state,geoid,refresh,name
30,Forrest,MS,28035,2021-12-14,ms_forrest
148,Noxubee,MS,28103,2021-12-14,ms_noxubee
155,Calhoun,MS,28013,2021-12-14,ms_calhoun
176,Leake,MS,28079,2021-12-14,ms_leake
205,Monroe,FL,12087,2021-12-14,fl_monroe
...,...,...,...,...,...
3060,Washington,FL,12133,2021-12-14,fl_washington
3061,Suwannee,FL,12121,2021-12-14,fl_suwannee
3062,Volusia,FL,12127,2021-12-14,fl_volusia
3063,Walton,FL,12131,2021-12-14,fl_walton


time: 10.6 ms (started: 2021-12-21 01:42:20 -05:00)


In [38]:
len(set(county_dfs.name))

219

time: 1.72 ms (started: 2021-12-21 01:43:54 -05:00)


In [39]:
county_dfs.name

30         ms_forrest
148        ms_noxubee
155        ms_calhoun
176          ms_leake
205         fl_monroe
            ...      
3060    fl_washington
3061      fl_suwannee
3062       fl_volusia
3063        fl_walton
3064      fl_st-johns
Name: name, Length: 219, dtype: object

time: 2.99 ms (started: 2021-12-21 01:52:51 -05:00)


In [40]:
county_dfs

,county,state,geoid,refresh,name
30,Forrest,MS,28035,2021-12-14,ms_forrest
148,Noxubee,MS,28103,2021-12-14,ms_noxubee
155,Calhoun,MS,28013,2021-12-14,ms_calhoun
176,Leake,MS,28079,2021-12-14,ms_leake
205,Monroe,FL,12087,2021-12-14,fl_monroe
...,...,...,...,...,...
3060,Washington,FL,12133,2021-12-14,fl_washington
3061,Suwannee,FL,12121,2021-12-14,fl_suwannee
3062,Volusia,FL,12127,2021-12-14,fl_volusia
3063,Walton,FL,12131,2021-12-14,fl_walton


time: 7.79 ms (started: 2021-12-21 02:00:50 -05:00)


In [33]:
len(county_dfs.county.unique())

191

time: 1.93 ms (started: 2021-12-21 01:43:00 -05:00)


In [25]:
updated_states = []
for state_df in state_dfs:
    STATE_FIPS_LENGTH = 2
    statefips = state_df.iloc[0].geoid[:STATE_FIPS_LENGTH]
    updated_states.append(statefips)

updated_states

['06', '08', '12', '19', '16', '27', '28', '38', '42', '48', '51', '55']

time: 4.27 ms (started: 2021-12-16 21:16:23 -05:00)


In [26]:
state_df

,county,state,geoid,refresh,name
1244,Milwaukee,WI,55079,2021-11-28,wi_milwaukee


time: 7.41 ms (started: 2021-12-16 21:16:25 -05:00)


## MAIN

In [34]:
updated_counties = []

for index, row in state_df.iterrows():
    county, geoid, name = row['county'], row['geoid'], row['name']
    
#     # 1. unzip county name ~/regrid-bucket/name.geojson.zip (in boss-geo-data)
#     unzipCmd = f"unzip -o -d /home/nhat/regrid-bucket/ /home/nhat/regrid-bucket/{name}.geojson.zip".split()
#     run_unzip_res =  subprocess.run(unzipCmd, check=True, capture_output=True, text=True)
#     print(f'{run_unzip_res=}')
    
#     # 2. MAIN FUNC: slow: ijson stream parse ~/regrid-bucket/county.json into ~/jq-jsons/
    jq_json = stream_parse(name)
    updated_counties.append(jq_json)
    
# 3. run parcelNumAddr.py to joint point-in-polygon and upload to ES based on given geoid/
print(f'{statefips=}, {updated_counties=}')
# MAIN FUNC: slow: getting numAddr and upload to ES
# es_df = parcel_addr(statefips, updated_counties)

# AFTER UNZIPPING geojson.zip, generate script to re-tile mbtiles
# 4. MAIN FUNC: slow: re-tile mbtiles for entire states, for each state in updated_states
# ['06', '08', '12', '19', '16', '27', '28', '38', '42', '48', '51', '55']
tippecanoe_cmdFile = f'/home/nhat/mbtiles/update-commands.txt'
gen_tippecanoe_commands(updated_states, tippecanoe_cmdFile)

# 5. postgis: 
# organize by state_abbrev (parcels_ak, parcels_al, etc.)
# FIRST: add row filename_stem (from a local saved dict mapping) for all records
# remove rows from state table with matching (county) filename_stem
# can handle EXCEPTION: geoid = 26163 {mi_wayne_detroit and mi_wayne} :: filename_stem

Reading input file name='wi_milwaukee' at: input_file='/home/nhat/regrid-bucket/wi_milwaukee.json'
Wrote/updated county name='wi_milwaukee' json to output_file='/home/nhat/jq-jsons/wi_milwaukee.json'
statefips='55', updated_counties=['/home/nhat/jq-jsons/wi_milwaukee.json']
time: 1min 12s (started: 2021-12-16 21:19:03 -05:00)


In [51]:
pd.DataFrame([[1,2], [3,4]], columns=['a','b'])

,a,b
0,1,2
1,3,4


time: 14.4 ms (started: 2021-12-18 18:53:17 -05:00)


In [ ]:
for json in updated_counties:
    filename_stem = Path(json).stem
    

In [ ]:

# AFTER: 4.A External separate script: 
# cronjob conda activate py39 && python3 verse.py && bash ~/mbtiles/parallel-update.sh && docker restart tileserver


In [ ]:
# AFTER: 5.A EXTERNAL separate script:
# # RESTART martin after finish updating to POSTGIS!

In [ ]:
# for updated_statefips in updated_states:
updated_statefips = updated_states[-1]
state_abbrv = STATE_FIPS_DICT_52[updated_statefips].lower()
state_all_counties = glob(f"{REGRID_BUCKET_DIR}{state_abbrv}_*.json")

time: 519 ms (started: 2021-12-16 21:21:57 -05:00)


#### Experiment with groupby

In [434]:
# df = pd.DataFrame(data={'a': [1, 2, 3 , 2, 3, 3], 'b': [10, 20, 20, 10, 10, 30], 'c': [13, 13 , 13, 13 , 13, 13]})
# # https://stackoverflow.com/a/50866760
# dfs = [x for _, x in df.groupby('b')]
# dfs

time: 256 µs (started: 2021-12-15 05:26:18 +00:00)


# TOP

In [28]:
def parcel_addr(statefips, updated_counties):
    STATE_ABBRV = STATE_FIPS_DICT_52[statefips]
    state_abbrv = STATE_ABBRV.lower()
    
    # assemble state dataframe from county jsonl
    frames = []
    for jsonl in updated_counties:
        df = pd.read_json(jsonl)
        if not df.empty:
            df['cbg'] = df['cbg'].astype(str).str.zfill(BLOCKGROUP_FIPS_LENGTH)
            df['func'] = (df['func'].fillna(0) // 1000).astype(int)
            frames.append(df)

    state_parcels = pd.concat(frames)
    print(f"{state_parcels.shape=}")

    # READING TIGER SHP: NY: takes 25s, about 15 parcels per census block
    tiger_url = "https://www2.census.gov/geo/tiger/TIGER2019/TABBLOCK/tl_2019_{}_tabblock10.zip".format(statefips) 
    print(f'Reading Tiger shp {statefips=} {STATE_ABBRV=}')
    state_blocks = gp.read_file(tiger_url)
    state_blocks = state_blocks[['GEOID10', 'geometry']]
    state_blocks = state_blocks.rename({"GEOID10": "GEOID"}, axis = 1)
    state_blocks['cbg'] = state_blocks['GEOID'].str[:BLOCKGROUP_FIPS_LENGTH]
    print(f"{state_blocks.shape=}")
    
    # NY= 2min: long time
    state_parcels =  gp.GeoDataFrame(state_parcels, geometry=gp.points_from_xy(state_parcels.lon, state_parcels.lat, crs=state_blocks.crs)) 
    # BIG TIME BOTTLENECK: NY= 4min: also long time
    state_parcels = gp.sjoin(state_parcels, state_blocks, how='left', predicate='within')
    

    # DISOWNED PARCELS: parcels with (lat, lon) not found within state boundaries i.e. parcels w/o GEOID!
    disowned_parcels = state_parcels['GEOID'].isnull().sum()
    print(f"Number of disowned parcels: {disowned_parcels=}, {(disowned_parcels/state_parcels.shape[0] * 100) :.5f}%")
    # DIFF_CBG
    diff_cbg = state_parcels.loc[ state_parcels['cbg_left'] != state_parcels['cbg_right']]
    diff_cbg.shape # (61580, 8)
    print(f"Percentage of diff_cbg: {(diff_cbg.shape[0] / len(state_parcels) * 100):.2f} %")
    
    # only need GEOID and func from now on
    state_parcels = state_parcels[['GEOID', 'func']].copy()
    
    # parcel number addresses per census block
    state_blocks['parcelNumAddr'] = state_blocks['GEOID'].map(state_parcels['GEOID'].value_counts()).fillna(value=np.nan)
    state_blocks = state_blocks.dropna(subset=['parcelNumAddr'])
    state_blocks['parcelNumAddr'] = state_blocks['parcelNumAddr'].astype(int)

    def cat_func(func):
        if func == 1:               return "parcelNumResi"
        elif func == 9:             return "parcelNumAgri"
        elif func in (2,3,5,7,8):   return "parcelNumCommer"
        elif func in (4,6):         return "parcelNumInfra"
        return "parcelNumNull"

    state_parcels['func'] = state_parcels['func'].apply(cat_func)
    state_parcels = state_parcels.groupby(['GEOID', 'func']).size().unstack(level=-1)
    # Some individual parcelNum cols might be missing; fill them up in this case
    parcelNumCols = ['parcelNumAgri', 'parcelNumCommer',  'parcelNumInfra', 'parcelNumNull', 'parcelNumResi']
    for col in parcelNumCols:
        if col not in state_parcels.columns:
            state_parcels[col] = 0

    # PROCESS the df to UPLOAD TO ES!
    es_df = state_blocks.merge(state_parcels, how='left', left_on="GEOID", right_index=True)
    del state_blocks
    del state_parcels
    es_df = es_df[['GEOID', 'parcelNumAddr', 'parcelNumAgri', 'parcelNumCommer',  'parcelNumInfra', 'parcelNumNull', 'parcelNumResi']]
    es_df['parcelNumAgri'] = es_df['parcelNumAgri'].fillna(0).astype(int)
    es_df['parcelNumCommer'] = es_df['parcelNumCommer'].fillna(0).astype(int)
    es_df['parcelNumInfra'] = es_df['parcelNumInfra'].fillna(0).astype(int)
    es_df['parcelNumNull'] = es_df['parcelNumNull'].fillna(0).astype(int)
    es_df['parcelNumResi'] = es_df['parcelNumResi'].fillna(0).astype(int)

    # # NOTE: [another possibly simpler approach] can also just get the parcelNum individual fields first, then sum up to get parcelNumAddr
    parcelNum_sum_check = es_df[['parcelNumAddr', 'parcelNumAgri', 'parcelNumCommer', 'parcelNumInfra', 'parcelNumNull', 'parcelNumResi']].sum()
    if (parcelNum_sum_check[0] - sum(parcelNum_sum_check[1:]) != 0):
        print("DOUBLE CHECK parcelNum fields: got wrong result for breaking down parcelNumAddr into smaller fields")
    else:
        print("parcelNum: Sum checked is OK")
    

    def faster_upload_df(df, index_name):
        idCol = 'GEOID'
        # updating_dict: a dict of all the data to be updated
        updating_dict = df.to_json(orient = "records")
        updating_dict = json.loads(updating_dict)

        # create an actiondf for all actions
        actiondf = df[[idCol]].copy()
        actiondf['_op_type'] = 'update'
        actiondf['_index'] = index_name
        actiondf['_id'] = actiondf[idCol]
        actiondf['doc_as_upsert'] = True
        actiondf['doc'] = updating_dict 
        actiondf.drop(idCol, axis = 1, inplace = True)

        udf = actiondf.to_json(orient = 'records')
        # all actions
        action_list = json.loads(udf) 
        print(f"Start uploading {len(df)} records to {index_name}")
        helpers.bulk(es, action_list)
        print(f"Completed uploading {len(df)} records to {index_name}")

    # # FOR TESTING
    # test = es_df[['GEOID', 'parcelNumAddr', "parcelNumAgri"]].copy()
    # test['parcelNumAddr'] = 44
    # test["parcelNumResi"] = 55
    # index_name = 'nhat_test_1'
    # faster_upload_df(test, index_name) # 45seconds! 16x faster than previous upload method
    
    # null check before uploading
    if sum(es_df.isnull().sum()) == 0:
        # ACTUAL UPLOAD
        index_name = f'bossdata{statefips}'
        faster_upload_df(es_df, index_name) 
    else:
        # save to file
        temp = f'/home/nhat/temp/numAddr_{statefips}.json'
        es_df.to_json(temp, orient='records')
        print(f"Error: es_df still has {sum(es_df.isnull().sum())} null values; saved es_df to {temp}.")
    
    return es_df

time: 4.83 ms (started: 2021-12-16 21:17:36 -05:00)
